In [110]:
# EfficientDet-D4 on Kaggle T4×2 (Group 1)

# Cell 0: 环境检查 & 依赖安装
import torch; print("CUDA:", torch.cuda.is_available(), "GPUs:", torch.cuda.device_count())
!pip install effdet pycocotools albumentations scikit-learn matplotlib tqdm


CUDA: True GPUs: 2


In [111]:
# Cell 1: 导入常用库 & 处理 Mixup 导入
import os, glob, math, json
import torch
import numpy as np
import pandas as pd
import albumentations as A
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from effdet import get_efficientdet_config, create_model, DetBenchTrain, DetBenchPredict
# 尝试导入 Mixup
try:
    from effdet.data.transforms import Mixup
    mixup_fn = Mixup(mixup_alpha=1.0, prob=0.5, num_classes=9)
    print("✅ Mixup enabled")
except ImportError:
    Mixup = None
    mixup_fn = None
    print("⚠️ Mixup not available, skipping")
from torchvision.ops import nms
from tqdm.auto import tqdm
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt


⚠️ Mixup not available, skipping


In [112]:
# Cell 2: 数据统计（验证不平衡）
label_files = glob.glob('/kaggle/input/**/train/labels/*.txt', recursive=True)
counts = {i:0 for i in range(9)}
for lf in label_files:
    with open(lf) as f:
        for line in f:
            cls = int(line.split()[0])
            counts[cls] += 1
df = pd.DataFrame({
    'class_id': list(counts.keys()),
    'count': list(counts.values())
})
print("Train distribution:\n", df)


Train distribution:
    class_id  count
0         0    123
1         1     71
2         2    640
3         3    159
4         4    141
5         5    266
6         6    127
7         7   3169
8         8   1610


In [113]:
# Cell 3: Dataset + Augmentations + Sampler —— 最终可跑版

import glob, numpy as np, torch, matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
import albumentations as A

# 1. 固定输入尺寸
IMG_SIZE = 1024

# 2. Dataset
class TrafficDataset(Dataset):
    def __init__(self, root, split='train', transforms=None):
        self.img_paths = sorted(glob.glob(f"{root}/{split}/images/*.jpg"))
        self.label_paths = [
            p.replace(f"/{split}/images/", f"/{split}/labels/").replace('.jpg', '.txt')
            for p in self.img_paths
        ]
        self.transforms = transforms

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        img = plt.imread(self.img_paths[idx])
        h, w = img.shape[:2]

        # 读取 YOLO 标签
        boxes, labels = [], []
        with open(self.label_paths[idx]) as f:
            for line in f:
                if not line.strip():
                    continue
                c, xc, yc, bw, bh = map(float, line.split())
                x1 = (xc - bw / 2) * w
                y1 = (yc - bh / 2) * h
                boxes.append([x1, y1, bw * w, bh * h])
                labels.append(int(c))
        boxes  = np.array(boxes)  if boxes  else np.zeros((0, 4))
        labels = np.array(labels) if labels else np.zeros((0,), dtype=int)

        # Albumentations Resize + Aug
        if self.transforms:
            aug = self.transforms(image=img, bboxes=boxes, category_id=labels)
            img, boxes, labels = aug['image'], np.array(aug['bboxes']), np.array(aug['category_id'])

        # 转 Tensor
        img_t    = torch.tensor(img).permute(2, 0, 1).float() / 255.0
        boxes_t  = torch.tensor(boxes,  dtype=torch.float32)
        labels_t = torch.tensor(labels, dtype=torch.int64)
        num_pos  = torch.tensor([labels_t.size(0)], dtype=torch.int64)

        target = {
            'boxes': boxes_t,
            'labels': labels_t,
            'label_num_positives': num_pos,
            'box_num_positives':   num_pos,
        }
        return img_t, target

# 3. Augmentations
train_aug = A.Compose(
    [
        A.Resize(IMG_SIZE, IMG_SIZE),
        A.RandomBrightnessContrast(p=0.5),
        A.HorizontalFlip(p=0.5),
        #A.RandomRotate90(p=0.0),
    ],
    bbox_params=A.BboxParams(format='coco', label_fields=['category_id'])
)
val_aug = A.Compose(
    [A.Resize(IMG_SIZE, IMG_SIZE)],
    bbox_params=A.BboxParams(format='coco', label_fields=['category_id'])
)

# 4. DataSet
dataset_root = '/kaggle/input/0713-hackathon/dataset'
train_ds = TrafficDataset(dataset_root, 'train', transforms=train_aug)
val_ds   = TrafficDataset(dataset_root, 'val',   transforms=val_aug)
print(f"Train images: {len(train_ds)}, Val images: {len(val_ds)}")

# 5. 类别计数（来自 Cell 2）
# counts = {0:123, 1:71, 2:640, 3:159, 4:141, 5:266, 6:127, 7:3169, 8:1610}

# 6. 采样权重
image_weights, default_weight = [], 1.0
for lp in train_ds.label_paths:
    with open(lp) as f:
        lines = [l for l in f.read().splitlines() if l.strip()]
    if not lines:
        image_weights.append(default_weight)
    else:
        cls_list = [int(l.split()[0]) for l in lines]
        avg_cnt  = np.mean([counts[c] for c in cls_list])
        image_weights.append(1.0 / (np.sqrt(avg_cnt) + 1e-6))

assert len(image_weights) == len(train_ds)

# 7. 自定义 collate_fn：图像 stack，targets 保持列表
def collate_fn(batch):
    imgs, targets = list(zip(*batch))   # imgs: tuple(Tensor)  targets: tuple(dict)
    imgs = torch.stack(imgs, 0)         # B,3,H,W
    return imgs, list(targets)          # targets ⇒ list(dict)  不再做其他事

# 8. DataLoader
sampler = WeightedRandomSampler(image_weights, len(image_weights), replacement=True)
train_loader = DataLoader(train_ds, batch_size=1, sampler=sampler,
                          collate_fn=collate_fn, num_workers=4)
val_loader   = DataLoader(val_ds, batch_size=1, shuffle=False,
                          collate_fn=collate_fn, num_workers=4)

print("train_loader batches:", len(train_loader), "val_loader batches:", len(val_loader))


Train images: 564, Val images: 72
train_loader batches: 564 val_loader batches: 72


In [114]:
# Cell 4 ▸ EfficientDet-D4 + Optimizer + Scheduler （最终版）
import math, torch
from effdet import create_model, DetBenchTrain, get_efficientdet_config

# 1⃣ 配置
cfg = get_efficientdet_config('tf_efficientdet_d4')
cfg.num_classes = 9                    # 9 类

# 2⃣ 基础网络
base = create_model(cfg, bench_task='train', pretrained=True)

# 3⃣ DetBenchTrain（关键：create_labeler=True ）
model = DetBenchTrain(
    base, cfg,
    create_labeler=True          # ✅ 强制创建 anchor_labeler
).cuda()                         # 单 GPU，显存最稳

# ── 如果显存允许再并行，否则保持单卡 ──
# model = torch.nn.DataParallel(model)

# 4⃣ 优化器
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)

# 5⃣ Scheduler：1 epoch Warm-up ➜ Cosine
max_epochs = 20
def warmup_cosine(ep):
    return (ep + 1) if ep < 1 else 0.5 * (1 + math.cos(math.pi * (ep - 1) / (max_epochs - 1)))
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=warmup_cosine)

# 6⃣ AMP Scaler
scaler = torch.cuda.amp.GradScaler()

print("✅ EfficientDet-D4 Bench (labeler on) — 单 GPU, AMP ready")


KeyError: {'name': 'tf_efficientdet_d4', 'backbone_name': 'tf_efficientnet_b4', 'backbone_args': {'drop_path_rate': 0.2}, 'backbone_indices': None, 'image_size': [1024, 1024], 'num_classes': 9, 'min_level': 3, 'max_level': 7, 'num_levels': 5, 'num_scales': 3, 'aspect_ratios': [[1.0, 1.0], [1.4, 0.7], [0.7, 1.4]], 'anchor_scale': 4.0, 'pad_type': 'same', 'act_type': 'swish', 'norm_layer': None, 'norm_kwargs': {'eps': 0.001, 'momentum': 0.01}, 'box_class_repeats': 4, 'fpn_cell_repeats': 7, 'fpn_channels': 224, 'separable_conv': True, 'apply_resample_bn': True, 'conv_bn_relu_pattern': False, 'downsample_type': 'max', 'upsample_type': 'nearest', 'redundant_bias': True, 'head_bn_level_first': False, 'head_act_type': None, 'fpn_name': None, 'fpn_config': None, 'fpn_drop_path_rate': 0.0, 'alpha': 0.25, 'gamma': 1.5, 'label_smoothing': 0.0, 'legacy_focal': False, 'jit_loss': False, 'delta': 0.1, 'box_loss_weight': 50.0, 'soft_nms': False, 'max_detection_points': 5000, 'max_det_per_image': 100,

In [ ]:
# Cell 5 — 训练 + 验证（单 GPU, AMP, 累积梯度, 混淆矩阵）

from torchvision.ops import box_iou
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from effdet import DetBenchPredict

# ---------- 评估函数 -------------------------------------------------------
def evaluate(model, loader, iou_thr=0.5):
    model.eval()
    pred_head = DetBenchPredict(model)             # 单 GPU 直接用本体

    y_true, y_pred = [], []
    with torch.no_grad():
        for imgs, targets in loader:               # imgs: (B,3,H,W)
            imgs = imgs.cuda(non_blocking=True)
            out  = pred_head(imgs)                 # tuple (boxes, scores, classes)

            for bi, (boxes, scores, classes) in enumerate(zip(*out)):
                gt_lbl, gt_box = targets[bi]['labels'], targets[bi]['boxes']
                if gt_lbl.numel() == 0:
                    continue
                # IoU 匹配
                ious = box_iou(boxes.cpu(), gt_box.cpu())
                matches = torch.max(ious, dim=0)
                for gi, (mx_iou, idx) in enumerate(zip(matches[0], matches[1])):
                    if mx_iou >= iou_thr:
                        y_true.append(gt_lbl[gi].item())
                        y_pred.append(classes[idx].cpu().item())

    cm = confusion_matrix(y_true, y_pred, labels=list(range(9)))
    return cm

# ---------- 训练主循环 -----------------------------------------------------
max_epochs   = 20
batch_size   = 1        # 与 DataLoader 一致
accum_iter   = 4        # 累积 4 步等效 batch =4
best_f1, bad = 0, 0

step = 0
for epoch in range(max_epochs):
    model.train()
    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{max_epochs}")
    for imgs, targets in pbar:
        # 搬到 GPU
        imgs = imgs.cuda(non_blocking=True)
        targets = [
            {k: (v.cuda(non_blocking=True) if torch.is_tensor(v) else v)
             for k, v in t.items()}
            for t in targets
        ]

        with torch.cuda.amp.autocast():
            loss, _, _ = model(imgs, targets)
            loss = loss / accum_iter            # ① 累积梯度

        scaler.scale(loss).backward()

        # ② 每 accum_iter 步更新一次
        if (step + 1) % accum_iter == 0:
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()

        pbar.set_postfix(loss=float(loss) * accum_iter,   # 还原显示真实 loss
                         lr=optimizer.param_groups[0]['lr'])
        step += 1
    scheduler.step()

    # ---------- 验证 ----------
    cm = evaluate(model, val_loader)
    disp = ConfusionMatrixDisplay(cm, display_labels=list(range(9)))
    disp.plot(include_values=False, cmap='Blues', xticks_rotation='vertical')
    plt.title(f'Confusion Matrix Epoch {epoch+1}')
    plt.show()

    # ---------- Macro-F1 ----------
    precision = np.diag(cm) / (cm.sum(axis=0) + 1e-9)
    recall    = np.diag(cm) / (cm.sum(axis=1) + 1e-9)
    f1_vals   = 2 * precision * recall / (precision + recall + 1e-9)
    f1        = np.mean(f1_vals)
    print(f"Epoch {epoch+1}:  macro-F1 = {f1:.4f}")

    # ---------- Early-stop & Checkpoint ----------
    if f1 > best_f1:
        best_f1, bad = f1, 0
        torch.save(model.state_dict(), 'best_d4.pth')
    else:
        bad += 1
        if bad >= 5:
            print("Early stopping ⏹️")
            break


In [ ]:
# Cell 6: 最佳模型加载 & Test 推理 + 生成 submission
model.load_state_dict(torch.load('best_d4.pth'))
predictor = DetBenchPredict(model.module)

test_imgs = sorted(glob.glob('/kaggle/input/**/test/images/*.jpg', recursive=True))
results = []
for img_path in tqdm(test_imgs, desc="Test Inference"):
    img = torch.tensor(plt.imread(img_path)).permute(2,0,1).float()/255.
    boxes, scores, classes = predictor([img.cuda()])
    # 按类计数
    cnts = [0]*9
    for c in classes.cpu(): cnts[c] += 1
    results.append([os.path.basename(img_path), ';'.join(map(str,cnts))])

# 写 CSV
import csv
with open('submission.csv','w', newline='') as f:
    w=csv.writer(f); w.writerow(['pic_name','results'])
    w.writerows(results)
print("Saved submission.csv")
